# 01 - Exploration des données

Ce notebook explore les données historiques électorales et démographiques de Paris.

## Sources
- **opendata.paris.fr** : résultats électoraux par bureau de vote
- **INSEE** : données démographiques par arrondissement

## Objectifs
1. Charger et valider les données
2. Explorer les tendances historiques
3. Analyser les corrélations démographie × vote

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import display

from paris_elections.data.loader import DataLoader
from paris_elections.config import SECTEURS, ARRONDISSEMENT_TO_SECTEUR

## 1. Initialisation du chargeur de données

In [ ]:
loader = DataLoader()

# Élections disponibles
print("Élections disponibles via l'API opendata.paris.fr :")
for key, dataset in loader.available_elections().items():
    print(f"  - {key}: {dataset}")

## 2. Chargement des données électorales

### 2.1 Municipales 2020 (T1)

In [ ]:
try:
    muni_2020 = loader.load_election('municipales_2020_t1')
    print(f"Municipales 2020 T1 : {len(muni_2020)} bureaux de vote")
    print(f"Colonnes : {list(muni_2020.columns)[:10]}...")
    display(muni_2020.head())
except Exception as e:
    print(f"Erreur de chargement (API peut être indisponible) : {e}")
    muni_2020 = None

### 2.2 Agrégation par secteur

In [ ]:
if muni_2020 is not None:
    try:
        muni_2020_sectors = loader.load_election_by_sector('municipales_2020_t1')
        display(muni_2020_sectors)
    except Exception as e:
        print(f"Erreur d'agrégation : {e}")

## 3. Données démographiques INSEE

In [ ]:
# Population par arrondissement
population = loader.load_population()
print(f"Population Paris : {population['population'].sum():,} habitants")
display(population)

In [ ]:
# Agrégation par secteur
pop_sectors = loader.load_population_by_sector()
display(pop_sectors)

## 4. Visualisation de la répartition démographique

In [ ]:
fig = px.bar(
    pop_sectors.sort_values('population', ascending=True),
    x='population',
    y='secteur',
    orientation='h',
    title='Population par secteur (2021)',
    labels={'population': 'Population', 'secteur': 'Secteur'},
)
fig.update_layout(template='plotly_white')
fig.show()

## 5. Mapping des secteurs post-réforme

In [ ]:
print("Structure des 17 secteurs post-réforme 2025 :")
print("="*50)
for secteur, arrondissements in SECTEURS.items():
    arr_str = ', '.join(f"{a}e" if a > 1 else "1er" for a in arrondissements)
    print(f"{secteur:15} : {arr_str}")

## 6. Historique des participations

In [ ]:
# Données de participation historiques (manuelles si API indisponible)
participation_historique = pd.DataFrame({
    'election': ['Municipales 2014 T1', 'Municipales 2014 T2',
                 'Municipales 2020 T1', 'Municipales 2020 T2',
                 'Présidentielle 2022 T1', 'Présidentielle 2022 T2',
                 'Européennes 2024'],
    'participation': [0.543, 0.573, 0.305, 0.411, 0.771, 0.745, 0.53],
})

fig = px.bar(
    participation_historique,
    x='election',
    y='participation',
    title='Participation électorale à Paris (historique)',
    labels={'participation': 'Taux de participation', 'election': 'Élection'},
)
fig.update_layout(template='plotly_white', yaxis_tickformat='.0%')
fig.show()

## 7. État du cache

In [ ]:
cache_status = loader.cache_status()
print(f"Entrées en cache : {len(cache_status)}")
for key, meta in cache_status.items():
    print(f"  - {key}: {meta.get('rows', '?')} lignes, source={meta.get('source', '?')}")

---

## Prochaines étapes

➡️ **Notebook 02** : Calibration du modèle de redressement  
➡️ **Notebook 03** : Simulation interactive  
➡️ **Notebook 04** : Comparaison de scénarios